In [1]:
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

%matplotlib inline
%load_ext autoreload
%autoreload 2

# Import modules created for this project. Modules are in ./project_modules folder
import project_modules.calibrate_camera as cc
import project_modules.image_process as ip
import project_modules.find_lanes as fl
import project_modules.process_video as pv

In [2]:
# Image frame process pipeline - each frame is processed with this function
def image_process_pipeline(img, imgpoints, objpoints):
    img = ip.undistort_image(img, imgpoints, objpoints)
    img = ip.color_threshold(img)
    img, Minv = ip.perspective_transform(img)    
    return img, Minv

# Function to pass into video processing
def process_image(img):
    processed_img, Minv = image_process_pipeline(img, imgpoints, objpoints) # Process each image frame
    
    # Find the lanes and radius of curvature & car position in each image
    out_img, left_fit, right_fit, left_lane_inds, right_lane_inds = fl.find_lanes(processed_img)
    left_curverad, right_curverad, left_fitx, right_fitx = fl.curvature_measurement(out_img, left_fit, right_fit, left_lane_inds, right_lane_inds)
    car_position = fl.vehicle_position(out_img, left_fit, right_fit)
    
    # Apply smoothing filter in case some fits are outliers.
    left_fitx = fl.line_smoothing(left_line, left_fitx, left_curverad)
    right_fitx = fl.line_smoothing(right_line, right_fitx, right_curverad)
    left_line.curve_list.append(left_curverad)  # For smoothing the curve output in image overlay
   
    # Draw found lane onto image
    result = pv.draw_lane_image(img, processed_img, left_fitx, right_fitx, Minv)
    result = pv.output_curvature_data(result,left_curverad,car_position, left_line)
    
    return result

In [3]:
# Calibrate camera
image_names = cc.get_calibration_image_names()
imgpoints, objpoints = cc.calibrate_camera(image_names)

In [4]:
# Create left and right line objects
left_line = fl.Line()
right_line = fl.Line()

# Process input video
vid_in = 'project_video.mp4'
vid_out = 'Advanced Lane Finding Output.mp4'
pv.process_video(vid_in, vid_out, process_image)

[MoviePy] >>>> Building video Advanced Lane Finding Output.mp4
[MoviePy] Writing video Advanced Lane Finding Output.mp4


100%|█████████████████████████████████████████████████████████████████████████████▉| 1260/1261 [12:50<00:00,  1.64it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: Advanced Lane Finding Output.mp4 

